In [3]:
import argparse
import os

import cv2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor

import fiftyone as fo
import fiftyone.utils.random as four
from fiftyone import ViewExpression as E
import loading_data_from_fiftyone_to_detectron2 as loading_data


# 模型加载
def load_model(config_file, model_weights):
    cfg = get_cfg()  # CfgNode
    # config参数详解：https://detectron2.readthedocs.io/en/latest/modules/config.html
    cfg.merge_from_file(config_file)  # 合并（加载）模型的配置文件（模型结构、优化器设置...）

    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10  # ROI的标签数量
    cfg.MODEL.WEIGHTS = model_weights
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # 设置ROI的得分的阈值

    predictor = DefaultPredictor(cfg)
    return predictor


# 模型推理
def inference(predictor, image_path):
    image_path = image_path.replace('\\', '\\\\')
    img = cv2.imread(image_path)
    print(image_path)
    img_h, img_w, _ = img.shape
    outputs = predictor(img)
    return outputs  # 在推理模式下，内置模型输出一个list[dict]，每个字典可能包含以下字段。https://detectron2.readthedocs.io/en/latest/tutorials/models.html?highlight=DefaultPredictor#model-output-format


# 将推理结果转为fiftyone格式
def detectron_to_fo(outputs, img_w, img_h):
    # 一张图片有多个缺陷
    instances = outputs['instances'].to('cpu')
    detections = []
    for pre_box, socre, label in zip(instances.pred_boxes, instances.scores,
                                     instances.pred_classes):  # pred_classes --> 列别索引：[0, num_categories)
        x1, y1, x2, y2 = pre_box
        bbox = [float(x1) / img_w, float(y1) / img_h, float(x2 - x1) / img_w, float(y2 - y1) / img_h]
        detection = fo.Detection(label=list(loading_data.category_mapping_zh.keys())[label], confidence=float(socre), bounding_box=bbox)  # label

        detections.append(detection)
    return fo.Detections(detections=detections)


# 验证操作
def evaluate(args, predictor):
    images_dir = r'E:\SC_search_longfaning\car-component-detect\dataset\car_parts\coco_format\images'
    annotations_file = r'E:\SC_search_longfaning\car-component-detect\dataset\car_parts\coco_format\annotations\annotations.json'

    # 加载自定义CoCo格式的数据集
    dataset_name = 'test5'
    if dataset_name in fo.list_datasets():
        car_parts_dataset = fo.load_dataset(dataset_name)
    else:
        car_parts_dataset = fo.Dataset.from_dir(
            name=dataset_name,
            data_path=images_dir,
            labels_path=annotations_file,
            dataset_type=fo.types.COCODetectionDataset
        )
    
    # 划分数据集
    four.random_split(car_parts_dataset, {'train': 0.75, 'test': 0.15, 'val': 0.1})

    # 拿到val数据集
    val_view = car_parts_dataset.match_tags("val")
    dataset_dicts = loading_data.get_data_dicts_from_fiftyone(val_view)

    predictions = {}
    # 遍历
    for data in dataset_dicts:
        img_w = data["width"]
        img_h = data["height"]
        img_path = data['file_name']

        # 推理
        outputs = inference(predictor, img_path)

        # 将推理结果转为fiftyone格式
        detections = detectron_to_fo(outputs, img_w, img_h)
        predictions[data['image_id']] = detections

    # 在fiftyone的json文件内容追加predictions字段
    car_parts_dataset.set_values(field_name="predictions", values=predictions,
                                 key_field="id")  # key_field="id" 指定了字典的键对应的是fiftyone的数据集样本的 id 字段
    return car_parts_dataset


# 汽车零件检测
# 一张一张图片进行检测
def detect(args, predictor):
    # 模型推理
    outputs = inference(predictor, args.image)

    img = cv2.imread(args.image)
    img_h, img_w = img.shape[:2]

    # 将推理结果转为fiftyone格式
    detections = detectron_to_fo(outputs, img_w, img_h)

    # 创建Sample对象
    sample = fo.Sample(filepath=args.image)

    sample["detections"] = detections
    dataset_name = "car_parts_detection"
    if dataset_name in fo.list_datasets():
        # 如果存在dataset_name，直接加载就好
        car_parts_dataset = fo.load_dataset(dataset_name)
    else:
        # 创建数据集
        car_parts_dataset = fo.Dataset(name=dataset_name)
    # 将包含图像和检测结果的sample对象添加到数据集中
    car_parts_dataset.add_sample(sample)  # type(sample) --> class:`fiftyone.core.sample.Sample
    return car_parts_dataset


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description="Detectron2 Inference and FiftyOne Visualization")
    parser.add_argument('--config', type=str, default='COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml',
                        help='Path to the config file')
    parser.add_argument('--weights', type=str, default=r'C:\Users\ASUS\Desktop\test\model_0012997.pth',
                        help='Path to the model weights')
    parser.add_argument('--dataset_name', type=str, default='test5', help='dataset name to the fiftyone')
    parser.add_argument('--operation', type=str, default='detect', help='operation choose evaluate or detect')
    parser.add_argument('--image', type=str, default='', help='Path to the image')

#     args = parser.parse_args()
    args =parser.parse_known_args()[0]
    config_file = model_zoo.get_config_file(args.config)
    # model_weights = model_zoo.get_checkpoint_url(args.weights)
    model_weights = args.weights
    # 实例化对象
    predictor = load_model(config_file, model_weights)

    if args.operation == 'evaluate':
        # 评估模型效果
        car_parts_dataset = evaluate(args, predictor)
    elif args.operation == 'detect':
        image_dir = r'C:\Users\ASUS\Desktop\test\Images'
        for image_name in os.listdir(image_dir):
            image_path = os.path.join(image_dir,image_name)
            args.image = image_path
            # 一张一张进行检测
            car_parts_dataset = detect(args, predictor)
    
#     # 评估模型的质量
#     car_parts_dataset.evaluate_detections(pred_field='predictions', gt_field='detections', eval_key="eval")  # pred_field 模型预测结果的字段，gt_field真实标签字段，eval_key评估结果保存的字段

#     # 向sample的tags字段添加内容
#     for sample in car_parts_dataset.iter_samples(autosave=True, batch_size=100):
#         for detection in sample.detections.detections:
#             detection['tags'].append(detection['supercategory'])
    
    # 启动web
    session = fo.launch_app(car_parts_dataset)
    
    session.view = car_parts_dataset.to_evaluation_patches("eval")
    session.freeze()
#     session.wait()

C:\\Users\\ASUS\\Desktop\\test\\Images\\2020_03_07_06_04_52_047.jpg
C:\\Users\\ASUS\\Desktop\\test\\Images\\2020_03_07_06_04_56_939.jpg
C:\\Users\\ASUS\\Desktop\\test\\Images\\2020_03_07_06_05_07_687.jpg
C:\\Users\\ASUS\\Desktop\\test\\Images\\2021_05_16_13_30_12.jpg
C:\\Users\\ASUS\\Desktop\\test\\Images\\2021_05_16_13_33_08.jpg
C:\\Users\\ASUS\\Desktop\\test\\Images\\2021_05_16_13_34_08.jpg
C:\\Users\\ASUS\\Desktop\\test\\Images\\2021_05_22_10_17_47.jpg
C:\\Users\\ASUS\\Desktop\\test\\Images\\2021_05_22_10_17_51.jpg
C:\\Users\\ASUS\\Desktop\\test\\Images\\2021_05_22_10_18_17.jpg
C:\\Users\\ASUS\\Desktop\\test\\Images\\2021_05_22_10_18_20.jpg
Connected to FiftyOne on port 5151 at localhost.
If you are not connecting to a remote session, you may need to start a new session and specify a port


ValueError: Dataset has no evaluation key 'eval'

In [2]:
# 启动web
session = fo.launch_app(car_parts_dataset)

# session.view = car_parts_dataset.to_evaluation_patches("eval")
session.freeze()

In [41]:
for sample in car_parts_dataset:
#     for detection in sample.detections.detections:
#         print(detection)
#         print('=================')
    sample.add_label_tag(sample.detections.detections[0].supercategory)
#         detection['tags'] = detection['supercategory']




AttributeError: Sample has no field 'add_label_tag'

In [51]:
sample = car_parts_dataset.first()
print(sample)
# sample.tags.append("new_tag")
# sample.save()

<Sample: {
    'id': '64c0dbb8e504c2998f45968d',
    'media_type': 'image',
    'filepath': 'E:\\SC_search_longfaning\\car-component-detect\\dataset\\car_parts\\coco_format\\images\\1_2020_01_11_21_36_02_642.jpg',
    'tags': ['train'],






    }>,




























































    }>,
    'predictions': None,
    'eval_tp': 0,
    'eval_fp': 0,
    'eval_fn': 3,
}>


In [47]:
for sample in car_parts_dataset:
    print(sample)
    break

<Sample: {
    'id': '64c0dbb8e504c2998f45968d',
    'media_type': 'image',
    'filepath': 'E:\\SC_search_longfaning\\car-component-detect\\dataset\\car_parts\\coco_format\\images\\1_2020_01_11_21_36_02_642.jpg',
    'tags': ['train'],






    }>,




























































    }>,
    'predictions': None,
    'eval_tp': 0,
    'eval_fp': 0,
    'eval_fn': 3,
}>


In [ ]:
# 添加标签标签
for sample in dataset:
    for label in sample.labels.values():
        label.add_label_tag("your-tag")

In [37]:
print(car_parts_dataset.first)
    

<bound method Dataset.first of Name:        test5
Media type:  image
Num samples: 4643
Persistent:  False
Tags:        []
Sample fields:
    id:          fiftyone.core.fields.ObjectIdField
    filepath:    fiftyone.core.fields.StringField
    tags:        fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    predictions: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    eval_tp:     fiftyone.core.fields.IntField
    eval_fp:     fiftyone.core.fields.IntField
    eval_fn:     fiftyone.core.fields.IntField>


In [38]:
for detection in result.detections:
    print(detection)

<Detection: {
    'id': '64c0dbb8e504c2998f459687',
    'attributes': {},
    'tags': 'bearing_interface',
    'label': 'right_angle_edge_defect',
    'bounding_box': [
        0.9324534161490683,
        0.7040816326530612,
        0.024844720496894408,
        0.07936507936507936,
    ],
    'mask': None,
    'confidence': None,
    'index': None,
    'supercategory': 'bearing_interface',
    'iscrowd': 0,
    'eval': 'fn',
    'eval_id': '',
}>
<Detection: {
    'id': '64c0dbb8e504c2998f459688',
    'attributes': {},
    'tags': 'bearing_interface',
    'label': 'connection_edge_defect',
    'bounding_box': [
        0.8532608695652174,
        0.5158730158730159,
        0.017857142857142856,
        0.0963718820861678,
    ],
    'mask': None,
    'confidence': None,
    'index': None,
    'supercategory': 'bearing_interface',
    'iscrowd': 0,
    'eval': 'fn',
    'eval_id': '',
}>
<Detection: {
    'id': '64c0dbb8e504c2998f459689',
    'attributes': {},
    'tags': 'bearing_int

In [17]:
for detection in result.detections:
    print(detection.supercategory)

bearing_interface
bearing_interface
bearing_interface


In [31]:
car_parts_dataset.tag_labels

<bound method SampleCollection.tag_labels of Name:        test5
Media type:  image
Num samples: 4643
Persistent:  False
Tags:        []
Sample fields:
    id:          fiftyone.core.fields.ObjectIdField
    filepath:    fiftyone.core.fields.StringField
    tags:        fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    predictions: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    eval_tp:     fiftyone.core.fields.IntField
    eval_fp:     fiftyone.core.fields.IntField
    eval_fn:     fiftyone.core.fields.IntField>

In [46]:
session = fo.launch_app(car_parts_dataset)

# session.view = car_parts_dataset.to_evaluation_patches("eval")
session.freeze()

In [52]:
for sample in car_parts_dataset:
    for detection in sample.detections.detections:
        detection['tags'].append(detection['supercategory'])

AttributeError: 'BaseDict' object has no attribute 'append'